In [ ]:
# pip install Spire.Doc

In [2]:
from spire.doc import *
from spire.doc.common import *
from dotenv import dotenv_values
import openai
from openai import OpenAI
import numpy as np

# Create a Document instance
document = Document()

C:\Users\hp\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\hp\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\hp\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
api_key = dotenv_values("../api_data.env")
openai.api_key = api_key['OPEN_AI_KEY']

In [4]:
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", openai.api_key))

## Loading a text document

In [5]:
document.LoadFromFile("input_data/Macquarie Group announces $A3.docx")

### Extracting Paragraphs from the text

In [6]:
# Create a list to store the extracted headings
headings = []
para = []
heading_flag = 0
sectionlst= []
# Iterate through all sections in the document
for i in range(document.Sections.Count):
    section = document.Sections[i]
    heading_flag = 0
    headng=''
    # Iterate through all paragraphs in each section
    for j in range(section.Paragraphs.Count):
        paragraph = section.Paragraphs[j]
        # Check if the style name of the paragraph contains Heading
        if paragraph.StyleName is not None and "Heading" in paragraph.StyleName:
            # Get the text of the paragraph and append it to the list
            headings.append(paragraph.Text)
        else:
            #if len(paragraph.Text) > 0:
            para.append(paragraph.Text)
                
            

# Save the content of the list to a text file
# with open("ExtractedHeadings.txt", "w") as file:
#     for heading in headings:
#         file.write(heading + "\n")

document.Close()

### Combining para chunks

In [7]:
# Step 1: Define your list
import pandas as pd
pars_lst = []

# Step 2: Define a function to extract chunks
def extract_chunks(lst, chunk_size):
    for i in range(0, len(lst), chunk_size):
        yield lst[i:i + chunk_size]

# Step 3: Extract chunks
chunk_size = 5
chunks = list(extract_chunks(para, chunk_size))

# Step 4: Convert chunks to DataFrames


for chu in chunks:
    pars_lst.append(' '.join(chu))
    

In [8]:
len(pars_lst)

10

### Extracting Embedding using text-embedding-3-small

In [9]:
EMBEDDING_MODEL = "text-embedding-3-small"

embeddings = []
embeddings_dic ={}
for batch in range(0, len(pars_lst)):
    print(batch)
    response = client.embeddings.create(model=EMBEDDING_MODEL, input=pars_lst[batch])
    
    for i, be in enumerate(response.data):
        assert i == be.index  # double check embeddings are in same order as input
    batch_embeddings = [e.embedding for e in response.data]
    embeddings.extend(batch_embeddings)
    embeddings_dic[pars_lst[batch]] = batch_embeddings
    

0
1
2
3
4
5
6
7
8
9


In [10]:
df = pd.DataFrame(embeddings_dic.items()).rename(columns = {0:'text',1:'embedding'})

In [11]:
#df.to_csv("embedding_Macquarie_Group_announces_$A3.csv")

### Testing the stored embedding and input query match

In [17]:
df = pd.read_csv('Embedding_df/embedding_Macquarie_Group_announces_$A3.csv').drop(columns = ['Unnamed: 0'])

In [18]:
df.head()

,text,embedding
0,"FY24 net profit of $A3,522 million, down 32% o...","[[0.026646699756383896, 0.009173554368317127, ..."
1,Bank Level 2 CET1 ratio 13.6% (Harmonised5: 18...,"[[0.010210090316832066, 0.041706670075654984, ..."
2,"Annuity-style activities, which are undertaken...","[[0.0324883796274662, 0.04309939965605736, 0.0..."
3,Assets under management at 31 March 2024 were ...,"[[0.05020987242460251, 0.02893919125199318, 0...."
4,Macquarie’s financial position exceeds the Aus...,"[[0.0031701396219432354, 0.024023061618208885,..."


In [27]:
import ast 
df['embedding'] = df['embedding'].apply(ast.literal_eval)

In [28]:
# models
EMBEDDING_MODEL = "text-embedding-3-small"
GPT_MODEL = "gpt-3.5-turbo"

In [29]:
# search function
from scipy import spatial 
def strings_ranked_by_relatedness(
    query: str,
    df: pd.DataFrame,
    relatedness_fn=lambda x, y: 1 - spatial.distance.cosine(x, y),
    top_n: int = 100
) -> tuple:
    """Returns a list of strings and relatednesses, sorted from most related to least."""
    query_embedding_response = client.embeddings.create(
        model=EMBEDDING_MODEL,
        input=query,
    )
    query_embedding = query_embedding_response.data[0].embedding
    strings_and_relatednesses = [
        (row["text"], relatedness_fn(query_embedding, row["embedding"]))
        for i, row in df.iterrows()
    ]
    strings_and_relatednesses.sort(key=lambda x: x[1], reverse=True)
    strings, relatednesses = zip(*strings_and_relatednesses)
    return strings[:top_n], relatednesses[:top_n]

In [30]:
# examples
strings, relatednesses = strings_ranked_by_relatedness("MAM contribution in profit", df, top_n=5)
for string, relatedness in zip(strings, relatednesses):
    print(f"{relatedness=:.2f}")
    display(string)

relatedness=0.56


'Assets under management at 31 March 2024 were $A938.3 billion2, up seven per cent from $A878.6 billion at 31 March 2023, largely due to favourable market movements, investments made by MAM Private Markets-managed funds and favourable foreign exchange movements, partially offset by assets no longer managed as a result of a reduction of co-investment management rights. MAM delivered a net profit contribution of $A1,208 million, down 48 per cent from $A2,342 million in FY23. The result was primarily driven by lower asset realisations in green investments and increased net expenditure in investments in green energy portfolio companies operating on a standalone basis with base and performance fees broadly in line. BFS delivered a net profit contribution of $A1,241 million, up three per cent from $A1,201 million in FY23. The result reflected growth in the loan portfolio and BFS deposits11, and credit impairment reversals primarily reflecting an improvement in the macroeconomic outlook; part

relatedness=0.50


'The geographic composition of income and the impact of foreign exchange Potential tax or regulatory changes and tax uncertainties Ms Wikramanayake said: “Macquarie remains well-positioned to deliver superior performance in the medium term with its diverse business mix across annuity-style and markets-facing businesses; deep expertise across diverse sectors in major markets with structural growth tailwinds; patient adjacent growth across new products and new markets; ongoing investment in our operating platform; a strong and conservative balance sheet; and a proven risk management framework and culture.” Where referenced in this document, total income is net operating income excluding Corporate items. Macquarie Asset Management (MAM) Private Markets Assets under Management (AUM) includes equity yet to deploy and equity committed to assets but not yet deployed.'

relatedness=0.47


'Annuity-style activities, which are undertaken by Macquarie Asset Management (MAM), Banking and Financial Services (BFS) and certain businesses in Commodities and Global Markets (CGM), generated a combined net profit contribution7 of $A3,014 million, down 27 per cent on FY23, which was primarily due to lower asset realisations in green investments and continued investment in the development of green energy portfolio companies in MAM. Markets-facing activities, which are undertaken by Macquarie Capital and most businesses in CGM, delivered a combined net profit contribution of $A3,699 million, down 40 per cent on FY23, which was notably characterised by exceptional levels of volatility in commodity markets that drove a record FY23 performance from CGM. Net operating income of $A16,887 million was down 12 per cent on FY23, while operating expenses of $A12,061 million were broadly in line with FY23. International income accounted for 66 per cent of Macquarie’s total income. The income ta

relatedness=0.40


'Bank Level 2 CET1 ratio 13.6% (Harmonised5: 18.7%); Leverage ratio 5.2% (Harmonised5: 5.9%); LCR 191%6; NSFR 115%6 Return on equity (ROE) 10.8%, compared with 16.9% in FY23; 2H24 annualised ROE 12.9% Final ordinary dividend of $A3.85 per share (40% franked), FY24 ordinary dividend of $A6.40 per share (40% franked), representing a 2H24 payout ratio of 70% and FY24 payout ratio of 70% Macquarie Group (ASX: MQG; ADR: MQBKY) today announced a net profit after tax attributable to ordinary shareholders of $A3,522 million for the year ended 31 March 2024 (FY24), down 32 per cent on the year ended 31 March 2023 (FY23). Profit for the half year ended 31 March 2024 (2H24) was $A2,107 million, up 49 per cent on the half year ended 30 September 2023 (1H24) and down 27 per cent on the half year ended 31 March 2023 (2H23). Macquarie Group Managing Director and Chief Executive Officer, Shemara Wikramanayake, said: “Despite ongoing economic uncertainty and subdued market conditions in many parts of t

relatedness=0.39


'FY24 net profit of $A3,522 million, down 32% on FY23; 2H24 net profit of $A2,107 million, up 49% on 1H24, down 27% on 2H23 International income 66% of total income1 in FY24 Assets under management of $A938.3 billion2 at 31 March 2024, up 7% from 31 March 2023 and up 5% from 30 September 2023 Financial position comfortably exceeds regulatory minimum requirements Group capital surplus of $A10.7 billion3,4'

### Building the code for gpt response using Embedding search

In [31]:
GPT_MODEL = "gpt-3.5-turbo"
import tiktoken  # for counting tokens
import os

In [32]:
def num_tokens(text: str, model: str = GPT_MODEL) -> int:
    """Return the number of tokens in a string."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))


def query_message(
    query: str,
    df: pd.DataFrame,
    model: str,
    token_budget: int
) -> str:
    """Return a message for GPT, with relevant source texts pulled from a dataframe."""
    strings, relatednesses = strings_ranked_by_relatedness(query, df)
    introduction = 'Use the below articles on the SOW of a Company to answer questions"'
    question = f"\n\nQuestion: {query}"
    message = introduction
    for string in strings:
        next_article = f'\n\nWikipedia article section:\n"""\n{string}\n"""'
        if (
            num_tokens(message + next_article + question, model=model)
            > token_budget
        ):
            break
        else:
            message += next_article
    return message + question


def ask(
    query: str,
    df: pd.DataFrame = df,
    model: str = GPT_MODEL,
    token_budget: int = 4096 - 500,
    print_message: bool = False,
) -> str:
    """Answers a query using GPT and a dataframe of relevant texts and embeddings."""
    message = query_message(query, df, model=model, token_budget=token_budget)
    if print_message:
        print(message)
    messages = [
        {"role": "system", "content": "You answer questions about the macquarie annual report news."},
        {"role": "user", "content": message},
    ]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    response_message = response.choices[0].message.content
    return response_message

In [33]:
ask('Difference between CGM and MAM contribution in profit by number')

'The difference between CGM and MAM contribution in profit is $1,199 million. \n\nCGM delivered a net profit contribution of $A3,213 million, while MAM delivered a net profit contribution of $A2,014 million. Therefore, the difference in profit contribution between CGM and MAM is $3,213 million - $2,014 million = $1,199 million.'

In [ ]:
# pip install win32com

In [ ]:
# https://learn.microsoft.com/en-us/office/vba/api/outlook.oldefaultfolders 
# https://learn.microsoft.com/en-us/office/vba/api/outlook.folder.name
# https://cookbook.openai.com/examples/using_tool_required_for_customer_service

### Outlook Mails

In [ ]:
# import win32com.client

# outlook = win32com.client.Dispatch("Outlook.Application",userName="abhishek.lohit01@outlook.com").GetNamespace("MAPI")

# inbox = outlook.GetDefaultFolder(3) # "6" refers to the index of a folder - in this case,
#                                     # the inbox. You can change that number to reference
#                                     # any other folder
# # Print the total number of items in the Inbox
# print(f"Total Inbox Items: {inbox.UnReadItemCount}")
# # messages = inbox.Items
# # message = messages.
# # body_content = message.body
# # print("body_content: ",body_content)

In [ ]:
# import re
# for message in inbox.Items:
#     #print(message.To)
#     message_dic = {}
#     if (message.SenderName !='') & (message.Subject!='') & (message.Body!=''):
#         message.Body = message.Body.lower()
#         if('microsoft teams' in message.Body):
#             pass
#         else:
#             # Regex pattern to match http and https URLs
#             pattern = r'http[s]?://\S+'

#             # Remove the URLs from the string
#             message.Body = re.sub(pattern, '', message.Body)
#             #print("Subject: ", message.Subject)
#             print("Body: ",message.Body)
#             #print("Importance: ", ["Low", "Normal", "High"][message.Importance])
#             print("--"*70)